### Assignment 10
다음과 같은 CNN 모델을 작성해보자.
* Input
  + Input type: torch.Tensor
  + Input shape: (?, 1, 28, 28)
    - 여러장의, 흑백, 28x28 size의 이미지라고 가정하자.
* Layers
  + Layer 1
    - Conv2d >> C: 32, Kernel size (필터 크기): 3, Stride: 1, Padding: 1
    - ReLU
    - MaxPool >> Kernel size: 2, Stride: 2
    - 입-출력 (?, 1, 28, 28) >> (?, 32, 14, 14)
  + Layer 2
    - Conv2d >> C: 64, Kernel size (필터 크기): 3, Stride: 1, Padding: 1
    - ReLU
    - MaxPool >> Kernel size: 2, Stride: 2
    - 입-출력 (?, 32, 14, 14) >> (?, 64, 7, 7)
  + Layer 3
    - Conv2d >> C: 128, Kernel size (필터 크기): 3, Stride: 1, Padding: 1
    - ReLU
    - MaxPool >> Kernel size: 2, Stride: 2, Padding: 1
    - 입-출력 (?, 64, 7, 7) >> (?, 128, 4, 4)
  + Layer 4
   - Linear >> input: 4x4x128 output: 625
   - ReLU
   - Dropout
   - 입-출력 (4x4x128) >> (625)
  + Layer 5
   - Linear >> input: 625 output: 10
   - Softmax (pytorch의 Cross Entropy Loss 함수를 사용하는 것을 감안한다)



In [3]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [2]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 Input shape=(?, 1, 28, 28)
        #    Conv     -> (?, 32, 28, 28)
        #    Pool     -> (?, 32, 14, 14)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # L2 Input shape=(?, 32, 14, 14)
        #    Conv      ->(?, 64, 14, 14)
        #    Pool      ->(?, 64, 7, 7)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # L3 Input shape=(?, 64, 7, 7)
        #    Conv      ->(?, 128, 7, 7)
        #    Pool      ->(?, 128, 4, 4)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
        
        # L4 Linear 4x4x128 inputs -> 625 outputs
        # Dropout 0.3
        self.fc1 = torch.nn.Sequential(
        torch.nn.Linear(128*4*4, 625, bias=True),
        torch.nn.ReLU(),
        torch.nn.Dropout(p=0.3))
        torch.nn.init.xavier_uniform_(self.fc1.weight)
             
        # L5 Linear 625 inputs -> 10 outputs
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC

        out = self.fc1(out)
        out = self.fc2(out)
        return out